# 기출 6

In [15]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [16]:
dat=pd.read_csv('test/travel_insurance2.csv')
len(dat)

63326

# 전처리

In [17]:
pre1=dat.drop(columns=['ProductName','NetSales'])

In [18]:
pre1['Gender']=np.where(pre1['Gender'].isna(), 
                        np.where((pre1['NO'] % 2) == 0, 'M', 'F'), pre1['Gender'])

In [19]:
pre1.Gender.value_counts()

M    31891
F    31435
Name: Gender, dtype: int64

In [20]:
basetable1=pre1.copy()

# Q1

In [21]:
# help(basetable1['Age'].between

In [22]:
basetable1['Age'].between(40, 49)  # inclusive=False

0        False
1        False
2        False
3        False
4         True
         ...  
63321    False
63322     True
63323    False
63324    False
63325    False
Name: Age, Length: 63326, dtype: bool

In [23]:
q1=basetable1[basetable1['Age'].between(40, 49) & (basetable1['Gender'] == 'F')]
q1['Destination'].value_counts(ascending=False)

SINGAPORE         1352
MALAYSIA           439
THAILAND           434
INDONESIA          362
AUSTRALIA          323
                  ... 
ECUADOR              1
MAURITIUS            1
CAYMAN ISLANDS       1
GUATEMALA            1
CHILE                1
Name: Destination, Length: 72, dtype: int64

# Q2

In [24]:
basetable1['DistributionChannel'].value_counts()

Online     62219
Offline     1107
Name: DistributionChannel, dtype: int64

In [25]:
q2_1=basetable1[basetable1['DistributionChannel'] == 'Online']['Duration']
q2_2=basetable1[basetable1['DistributionChannel'] == 'Offline']['Duration']

In [26]:
import scipy.stats as st

In [27]:
st.ttest_ind(q2_1, q2_2)

Ttest_indResult(statistic=-10.289600869594274, pvalue=8.211742470098141e-25)

# Q3

In [28]:
q3=basetable1.copy()

In [29]:
q3=pd.get_dummies(q3, columns=['Gender','DistributionChannel'], drop_first=True)

In [30]:
train=q3[(q3['NO'] % 3) != 0]
test=q3[(q3['NO'] % 3) == 0]

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
train.columns

Index(['NO', 'Agency', 'AgencyType', 'Claim', 'Duration', 'Destination',
       'Commision', 'Age', 'Gender_M', 'DistributionChannel_Online'],
      dtype='object')

In [33]:
var_list=['Duration', 'Commision', 'Age', 'Gender_M', 'DistributionChannel_Online' ]

In [34]:
logit=LogisticRegression(C=100000, random_state=1234, solver='newton-cg')
logit.fit(train[var_list], train['Claim'])

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1234, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
pred=logit.predict_proba(test[var_list])

In [37]:
pred=pd.DataFrame(pred, columns=['No', 'Yes'])
pred.head(2)

,No,Yes
0,0.978281,0.021719
1,0.976549,0.023451


In [38]:
pred['Lift'] = pred['Yes']/0.015

In [40]:
pred.sort_values(by='Yes', ascending=False).iloc[:100, -1].sum()

1344.1053333434793